# Set-up

In [0]:
# load packages
import requests
from bs4 import BeautifulSoup

In [0]:
# Define the URL of the site
base_site = "https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/2/"

In [3]:
# sending a request to the webpage
response = requests.get(base_site)
response.status_code

200

In [0]:
# get the HTML from the webpage
html = response.content

## Choosing a parser

### html.parser

In [0]:
# convert the HTML to a Beautiful Soup object
soup = BeautifulSoup(html, 'html.parser')

In [0]:
# Exporting the HTML to a file
with open('Rotten_tomatoes_page_2_HTML_Parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [0]:
# When inspecting the file we see that HTML element is closed at the begining -- it parsed incorrectly!
# Let's check another parser

### lxml

In [0]:
# convert the HTML to a BeatifulSoup object
soup = BeautifulSoup(html, 'lxml')

In [0]:
# Exporting the HTML to a file
with open('Rotten_tomatoes_page_2_LXML_Parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [0]:
# By first accounts of inspecting the file everything seems fine

### A word of caution

In [0]:
# Beautiful Soup ranks the lxml parser as the best one.

# If a parser is not explicitly stated in the Beautiful Soup constructor,
# the best one available on the current machine is chosen.

# This means that the same piece of code can give different results on different computers.

## Finding an element containing all the data

In [9]:
# Find all div tags on the webpage containing the information we want to scrape
divs = soup.find_all("div", {"class": "col-sm-18 col-full-xs countdown-item-content"})
divs

[<div class="col-sm-18 col-full-xs countdown-item-content">
 <div class="row countdown-item-title-bar">
 <div class="col-sm-20 col-full-xs" style="height: 100%;">
 <div class="article_movie_title" style="float: left;">
 <div><h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2></div>
 </div>
 </div>
 <div class="col-sm-4 col-full-xs" style="height: 100%;">
 <div class="countdown-index">#140</div>
 </div>
 </div>
 <div class="row countdown-item-details">
 <div class="col-sm-24">
 <div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>61.188% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="to

# Extracting the title, year and score of each movie

In [0]:
# The title, year and score of each movie are contained in the 'h2' tags

In [11]:
# for instance, let's explore the first div
divs[0].find("h2")

<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>

In [12]:
# Extracting all 'h2' tags
headings = [div.find("h2") for div in divs]
headings

[<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/equilibrium/">Equilibrium</a> <span class="subtle start-year">(2002)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">40%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/hero/">Hero</a> <span class="subtle start-year">(2004)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/1017666-road_house/">Road House</a> <span class="subtle start-year">(1989)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">39%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/unstoppable-2010/">Unstoppable</a> <span class="subtle st

In [13]:
# Inspecting the text inside the headings
[heading.text for heading in headings]

['Running Scared (1986)  60%',
 'Equilibrium (2002)  40%',
 'Hero (2004)  95%',
 'Road House (1989)  39%',
 'Unstoppable (2010)  86%',
 'Shaft (1971)  88%',
 'The Villainess (Ak-Nyeo) (2017)  84%',
 'Highlander (1986)  69%',
 'Die Hard 2 (1990)  68%',
 'National Treasure (2004)  46%',
 'The Protector (Tom yum goong) (Warrior King) (2005)  53%',
 'Revenge (2018)  92%',
 'El Mariachi (1993)  93%',
 'A Touch of Zen (1969)  96%',
 'Top Gun (1986)  54%',
 'Con Air (1997)  55%',
 'The Expendables 2 (2012)  68%',
 'The Mummy (1999)  60%',
 'Mr. & Mrs. Smith (2005)  60%',
 'Rush Hour (1998)  60%',
 'The Equalizer (2014)  59%',
 'Captain America: Civil War (2016)  91%',
 'Air Force One (1997)  76%',
 'Bloodsport (1988)  38%',
 'Blade (1998)  55%',
 'Bad Boys (1995)  43%',
 'Die Hard: With a Vengeance (1995)  52%',
 'The Running Man (1987)  63%',
 'Code of Silence (1985)  63%',
 "Shoot 'Em Up (2007)  67%",
 'Crank (2006)  61%',
 'Machete (2010)  72%',
 'Drive (2011)  92%',
 'Batman (1989)  72%',

In [14]:
# It does contain the info we want to extract
# However, we need to obtain the title, year and score separately
# Let's inspect one heading to see if there is a way to distinguish between them
headings[0]

<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>

In [0]:
# We notice that:

# The movie title is in the 'a' tag
# The year is in a 'span' with class 'start-year'
# The score is in a 'span' with class 'tMeterScore'

## Title

In [0]:
# Let's check all heading links
[heading.find('a') for heading in headings]

[<a href="https://www.rottentomatoes.com/m/13_assassins_2011/">13 Assassins</a>,
 <a href="https://www.rottentomatoes.com/m/full_contact/">Full Contact</a>,
 <a href="https://www.rottentomatoes.com/m/indiana_jones_and_the_last_crusade/">Indiana Jones and the Last Crusade</a>,
 <a href="https://www.rottentomatoes.com/m/kung_fu_hustle/">Kung Fu Hustle</a>,
 <a href="https://www.rottentomatoes.com/m/better_tomorrow/">A Better Tomorrow</a>,
 <a href="https://www.rottentomatoes.com/m/iron_man/">Iron Man</a>,
 <a href="https://www.rottentomatoes.com/m/the_night_comes_for_us/">The Night Comes For Us</a>,
 <a href="https://www.rottentomatoes.com/m/logan_2017/">Logan</a>,
 <a href="https://www.rottentomatoes.com/m/goldfinger/">Goldfinger</a>,
 <a href="https://www.rottentomatoes.com/m/1001280-assault_on_precinct_13/">Assault on Precinct 13</a>,
 <a href="https://www.rottentomatoes.com/m/wonder_woman_2017/">Wonder Woman</a>,
 <a href="https://www.rottentomatoes.com/m/chinese_connection/">Fist of

In [0]:
# Obtaining the movie titles from the links
movie_names = [heading.find('a').string for heading in headings]
movie_names

['13 Assassins',
 'Full Contact',
 'Indiana Jones and the Last Crusade',
 'Kung Fu Hustle',
 'A Better Tomorrow',
 'Iron Man',
 'The Night Comes For Us',
 'Logan',
 'Goldfinger',
 'Assault on Precinct 13',
 'Wonder Woman',
 'Fist of Fury (Jing wu men)',
 'Captain America: The Winter Soldier',
 'Oldboy',
 'The French Connection',
 'Furious 7',
 'La Femme Nikita (Nikita)',
 'Supercop',
 'Dirty Harry',
 'Live Die Repeat: Edge of Tomorrow',
 'X2: X-Men United',
 'The Fugitive',
 'Black Panther',
 'Inception',
 'Braveheart',
 'Minority Report',
 'Avengers: Endgame',
 'Dredd',
 'The Bourne Identity',
 'Ip Man',
 'Face/Off',
 'To Live and Die in L.A.',
 'The Dark Knight',
 'Mission: Impossible Ghost Protocol',
 'Fast Five',
 'Lethal Weapon',
 'The Rock',
 'RoboCop',
 'John Wick: Chapter 2',
 'Casino Royale',
 'Baby Driver',
 'Fist of Legend (Jing wu ying xiong)',
 'The Killer',
 'The Raid 2',
 'Enter the Dragon',
 'Commando',
 'First Blood',
 'Mission: Impossible Rogue Nation',
 'The Terminat

## Year

In [0]:
# Filtering only the spans containing the year
[heading.find("span", class_ = 'start-year') for heading in headings]

[<span class="subtle start-year">(2011)</span>,
 <span class="subtle start-year">(1992)</span>,
 <span class="subtle start-year">(1989)</span>,
 <span class="subtle start-year">(2005)</span>,
 <span class="subtle start-year">(2010)</span>,
 <span class="subtle start-year">(2008)</span>,
 <span class="subtle start-year">(2018)</span>,
 <span class="subtle start-year">(2017)</span>,
 <span class="subtle start-year">(1964)</span>,
 <span class="subtle start-year">(1976)</span>,
 <span class="subtle start-year">(2017)</span>,
 <span class="subtle start-year">(1972)</span>,
 <span class="subtle start-year">(2014)</span>,
 <span class="subtle start-year">(2005)</span>,
 <span class="subtle start-year">(1971)</span>,
 <span class="subtle start-year">(2015)</span>,
 <span class="subtle start-year">(1990)</span>,
 <span class="subtle start-year">(1996)</span>,
 <span class="subtle start-year">(1971)</span>,
 <span class="subtle start-year">(2014)</span>,
 <span class="subtle start-year">(2003)<

In [0]:
# Extracting the year string
years = [heading.find("span", class_ = 'start-year').string for heading in headings]
years

['(2011)',
 '(1992)',
 '(1989)',
 '(2005)',
 '(2010)',
 '(2008)',
 '(2018)',
 '(2017)',
 '(1964)',
 '(1976)',
 '(2017)',
 '(1972)',
 '(2014)',
 '(2005)',
 '(1971)',
 '(2015)',
 '(1990)',
 '(1996)',
 '(1971)',
 '(2014)',
 '(2003)',
 '(1993)',
 '(2018)',
 '(2010)',
 '(1995)',
 '(2002)',
 '(2019)',
 '(2012)',
 '(2002)',
 '(2010)',
 '(1997)',
 '(1985)',
 '(2008)',
 '(2011)',
 '(2011)',
 '(1987)',
 '(1996)',
 '(1987)',
 '(2017)',
 '(2006)',
 '(2017)',
 '(1994)',
 '(1989)',
 '(2014)',
 '(1973)',
 '(1985)',
 '(1982)',
 '(2015)',
 '(1984)',
 '(2000)',
 '(2003)',
 '(1994)',
 '(1994)',
 '(1994)',
 '(2014)',
 '(2001)',
 '(1987)',
 '(2007)',
 '(1990)',
 '(1982)',
 '(1995)',
 '(2012)',
 '(2018)',
 '(1981)',
 '(1986)',
 '(1992)',
 '(1999)',
 '(1991)',
 '(1988)',
 '(2015)']

In [0]:
years[0]

'(2011)'

### Removing the brackets

In [0]:
# One way to remove the brackets is to drop the first and last symbol of the string
years[0][1:-1]

'2011'

In [0]:
# However, this will break, if the format of the year is changed

In [0]:
# Alternativelly, we can do it with the help of the strip() method (this is robust)

# It removes leading and trailing symbols from a string
# By default, it removes whitespace, but we can specify other symbols to strip

In [0]:
# Removing '('
years[0].strip('(')

'2011)'

In [0]:
# Removing ')'
years[0].strip(')')

'(2011'

In [0]:
# Combining both
years[0].strip('()')

'2011'

In [0]:
# Updating years with stripped values
years = [year.strip('()') for year in years]
years

['2011',
 '1992',
 '1989',
 '2005',
 '2010',
 '2008',
 '2018',
 '2017',
 '1964',
 '1976',
 '2017',
 '1972',
 '2014',
 '2005',
 '1971',
 '2015',
 '1990',
 '1996',
 '1971',
 '2014',
 '2003',
 '1993',
 '2018',
 '2010',
 '1995',
 '2002',
 '2019',
 '2012',
 '2002',
 '2010',
 '1997',
 '1985',
 '2008',
 '2011',
 '2011',
 '1987',
 '1996',
 '1987',
 '2017',
 '2006',
 '2017',
 '1994',
 '1989',
 '2014',
 '1973',
 '1985',
 '1982',
 '2015',
 '1984',
 '2000',
 '2003',
 '1994',
 '1994',
 '1994',
 '2014',
 '2001',
 '1987',
 '2007',
 '1990',
 '1982',
 '1995',
 '2012',
 '2018',
 '1981',
 '1986',
 '1992',
 '1999',
 '1991',
 '1988',
 '2015']

In [0]:
# Converting all the strings to integers
years = [int(year) for year in years]
years

[2011,
 1992,
 1989,
 2005,
 2010,
 2008,
 2018,
 2017,
 1964,
 1976,
 2017,
 1972,
 2014,
 2005,
 1971,
 2015,
 1990,
 1996,
 1971,
 2014,
 2003,
 1993,
 2018,
 2010,
 1995,
 2002,
 2019,
 2012,
 2002,
 2010,
 1997,
 1985,
 2008,
 2011,
 2011,
 1987,
 1996,
 1987,
 2017,
 2006,
 2017,
 1994,
 1989,
 2014,
 1973,
 1985,
 1982,
 2015,
 1984,
 2000,
 2003,
 1994,
 1994,
 1994,
 2014,
 2001,
 1987,
 2007,
 1990,
 1982,
 1995,
 2012,
 2018,
 1981,
 1986,
 1992,
 1999,
 1991,
 1988,
 2015]